In [ ]:
import pandas as pd
import os

In [ ]:
file_paths = {
    "VDJdb_beta": vdjdb_beta_read_path,
    "McPAS_beta": mcpastcr_beta_read_path,
    "IEDB_beta": iedb_beta_read_path,
    "pMTnet_beta": pmtnet_beta_read_path,
    "VDJdb_paired": vdjdb_paired_read_path,
    "McPAS_paired": mcpastcr_paired_read_path,
    "IEDB_paired": iedb_paired_read_path
}

In [ ]:
# Alle Daten speichern
all_data = {}

# Daten einlesen und vorbereiten
for file_name, path in file_paths.items():
    if os.path.exists(path):
        try:
            df = pd.read_csv(path, sep=None, engine="python")

            # tcr_key erstellen
            if "TRA_CDR3" in df.columns:
                df["tcr_key"] = df["TRA_CDR3"].astype(str) + '_' + df["TRB_CDR3"]
            else:
                df["tcr_key"] = df["TRB_CDR3"]

            all_data[file_name] = df[["tcr_key", "Epitope"]].drop_duplicates()

            print(f"{file_name} geladen mit {len(df)} Einträgen.")

        except Exception as e:
            print(f"Fehler beim Verarbeiten der Datei {file_name}: {e}")
    else:
        print(f"Datei nicht gefunden: {file_name}")

# Analyse: Welcher Datensatz sollte im Testset sein?
for test_file_name, test_df in all_data.items():
    # Alle anderen Daten als Trainingsset
    train_df = pd.concat([data for name, data in all_data.items() if name != test_file_name]).drop_duplicates()

    # Erstelle Sets für schnelles Lookup
    seen_tcrs = set(train_df["tcr_key"])
    seen_epitopes = set(train_df["Epitope"])

    # TPP3-Definition: Weder TCR noch Epitope wurden im Trainingsset gesehen
    tpp3_pairs = test_df[~test_df["tcr_key"].isin(seen_tcrs) & ~test_df["Epitope"].isin(seen_epitopes)]

    # Ausgabe der Ergebnisse
    print(f"\n**Wenn {test_file_name} als Testset verwendet wird:**")
    print(f"  - TPP3-Paare im Testset: {len(tpp3_pairs)}")
    print(f"  - Gesamt Test-Paare: {len(test_df)}")